# Part 2 - Setting up the system, visualising the funnel and preparing the fun-metaD simulations

In this part of the tutorial I will show you how to set up the BioSimSpace system, parameterising the protein and the ligand, as well as defining the simulation box, adding water and ions. Then, we will define a the funnel and visualise it using NGLview. Finally, we will setup the directories for the fun-metaD simulation itself.

## Part 2.1 - setup

First, import BioSimSpace.

In [17]:
import BioSimSpace as BSS
import os

In order to parameterise the protein, you just need a tleap-ready protein PDB file. We will use Amber ff14sb forcefield.

In [3]:
protein = BSS.IO.readMolecules('input_files/protein.pdb')
protein_water = protein.getWaterMolecules()

protein_search = protein.search('not water')
protein = protein_search.getResult(0)

protein = BSS.Parameters.ff14SB(protein, water_model="tip3p").getMolecule()

Parameterise the ligand using GAFF2.

In [4]:
# parameterise the ligand
ligand = BSS.IO.readMolecules('input_files/ligand.mol2').getMolecule(0)
ligand = BSS.Parameters.gaff2(ligand,
                          net_charge=BSS.Parameters.formalCharge(ligand)).getMolecule()

Now we solvate the protein-ligand system using TIP3P water.

In [6]:
# Find the dimensions of the protein
box_min, box_max = protein.getAxisAlignedBoundingBox()

# Work out the box size from the difference in the coordinates.
box_size = [y - x for x, y in zip(box_min, box_max)]

# how much to pad each side of the protein (nonbonded cutoff = 10 A)
padding = 15 * BSS.Units.Length.angstrom

box_length = max(box_size) + 2*padding

cmplx = protein + ligand

solvated = BSS.Solvent.tip3p(molecule = cmplx.toSystem(),
                        box=3*[box_length])

Save the prepared structures.

In [7]:
BSS.IO.saveMolecules('input_files/solvated', solvated, ['PDB','RST7','PRM7'])

['/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/solvated.pdb',
 '/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/solvated.rst7',
 '/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/solvated.prm7']

## Part 2.2 - visualisation
Before we run any dynamics, let's visualise the funnel defined automatically by BSS.

In [8]:
# Load the system.
system = BSS.IO.readMolecules(["input_files/solvated.pdb"])

# Create the funnel parameters.
p0, p1 = BSS.Metadynamics.CollectiveVariable.makeFunnel(system)

# Define the collective variable.
funnel_cv = BSS.Metadynamics.CollectiveVariable.Funnel(p0, p1)

# Create a view.
view = BSS.Metadynamics.CollectiveVariable.viewFunnel(system, funnel_cv)

view.system()

ThemeManager()

NGLWidget(gui_style='ngl')

Lets check what the funnel looks with a larger radius.

In [10]:
# Load the system.
system = BSS.IO.readMolecules(["input_files/solvated.pdb"])

# Create the funnel parameters.
p0, p1 = BSS.Metadynamics.CollectiveVariable.makeFunnel(system)

# Define the collective variable.
funnel_cv = BSS.Metadynamics.CollectiveVariable.Funnel(p0, p1, width = 1.5*BSS.Units.Length.nanometer)

# Create a view.
view = BSS.Metadynamics.CollectiveVariable.viewFunnel(system, funnel_cv)

view.system()

NGLWidget(gui_style='ngl')

In case the funnel was assigned incorrectly, you can open the structure file and use lists of atom IDs to define the p0 and p1 points.

Here I will load the same structure file with the p0 and p1 points defined by me.

In [12]:
# Load the system.
system = BSS.IO.readMolecules(["input_files/solvated.pdb"])

# Create the funnel parameters.
p0, p1 = [2624], [584, 1307, 1474, 1887]

# Define the collective variable.
funnel_cv = BSS.Metadynamics.CollectiveVariable.Funnel(p0, p1)

# Create a view.
view = BSS.Metadynamics.CollectiveVariable.viewFunnel(system, funnel_cv)

view.system()

NGLWidget(gui_style='ngl')

# Part 2.3 - Equilibrate the system with OpenMM
Load the prepared system first.

In [14]:
solvated = BSS.IO.readMolecules(['input_files/solvated.rst7','input_files/solvated.prm7'])

As OpenMM runs extremely well on GPUs, we will use it throughout the tutorial to equilibrate and run fun-metaD simulations.

In [18]:
os.environ['CUDA_VISIBLE_DEVICES']="0"

In [19]:
# minimise
protocol = BSS.Protocol.Minimisation()
process = BSS.Process.OpenMM(solvated, protocol, platform='CUDA')

# Start the process in the background.
process.start()

# Wait for the process to finish.
process.wait()

Get the minimised system and save the structure.

In [20]:
minimised = process.getSystem()

BSS.IO.saveMolecules('input_files/minimised',minimised, ['PDB','RST7','PRM7'])

['/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/minimised.pdb',
 '/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/minimised.rst7',
 '/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/minimised.prm7']

Equilibrate (note the very short run time).

In [21]:
protocol = BSS.Protocol.Equilibration(runtime=0.01*BSS.Units.Time.nanosecond)
process = BSS.Process.OpenMM(minimised, protocol)

# Start the process in the background.
process.start()

# Wait for the process to finish.
process.wait()

Get the equilibrated system and save the structure

In [22]:
equilibrated = process.getSystem()

BSS.IO.saveMolecules('input_files/equilibrated',equilibrated, ['PDB','RST7','PRM7'])

['/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/equilibrated.pdb',
 '/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/equilibrated.rst7',
 '/home/dom/Projects/git_pages/BioSimSpaceTutorials/04_funnel_metad/input_files/equilibrated.prm7']

## Part 2.4 - Setup and run fun-metaD

As I showed in the visualisation part, select the p0, p1 points for the funnel definition.

In [35]:
p0, p1 = BSS.Metadynamics.CollectiveVariable.makeFunnel(equilibrated)

Write the funnel CV for the simulation. Additionally, we will set a lower upper_bound value.

In [36]:
new_upper_bound = BSS.Metadynamics.Bound(value=3.5*BSS.Units.Length.nanometer)

funnel_cv = BSS.Metadynamics.CollectiveVariable.Funnel(p0, p1, upper_bound = new_upper_bound)

Write the metadynamics protocol.

In [37]:
protocol = BSS.Protocol.Metadynamics(funnel_cv,
                                     runtime = 0.001*BSS.Units.Time.nanosecond,
                                     hill_height = 1.5*BSS.Units.Energy.kj_per_mol,
                                     restart_interval = 10000,
                                     bias_factor = 10)

Run the metadynamics simulation.

In [38]:
process = BSS.Process.OpenMM(equilibrated, protocol, platform='CUDA',work_dir='fun-metaD')

process.start()

process.wait()

Save the final structures.

In [39]:
finished = process.getSystem()

BSS.IO.saveMolecules('fun-metaD/final', finished, ['PDB','RST7','PRM7'])

TypeError: 'system' must be of type 'BioSimSpace.SireWrappers.System', 'BioSimSpace._SireWrappers.Molecule, 'BioSimSpace._SireWrappers.Molecules' or a list of 'BiSimSpace._SireWrappers.Molecule' types.